# Определение перспективного тарифа для телеком-компании

Вы аналитик компании «Мегалайн» — федерального оператора сотовой связи. Клиентам предлагают два тарифных плана: «Смарт» и «Ультра». Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег.
Вам предстоит сделать предварительный анализ тарифов на небольшой выборке клиентов. В вашем распоряжении данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. Нужно проанализировать поведение клиентов и сделать вывод — какой тариф лучше.

## Изучение данных из файлов

In [49]:
import pandas as pd
from IPython.display import display

users = pd.read_csv('./datasets/users.csv')
display(users.head())
users.info()

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart
3,1003,23,NaN,Москва,Белла,Белякова,2018-08-17,ultra
4,1004,68,NaN,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   age         500 non-null    int64 
 2   churn_date  38 non-null     object
 3   city        500 non-null    object
 4   first_name  500 non-null    object
 5   last_name   500 non-null    object
 6   reg_date    500 non-null    object
 7   tariff      500 non-null    object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


В таблице users 500 строк и 8 колонок. В каждой строчке таблицы информация о пользователи компании. Согласно документации:

- `user_id` — уникальный идентификатор пользователя;
- `first_name` — имя пользователя;
- `last_name` — фамилия пользователя;
- `age` — возраст пользователя (годы);
- `reg_date` — дата подключения тарифа (день, месяц, год);
- `churn_date` — дата прекращения пользования тарифом (если значение пропущено, то тариф ещё действовал на момент выгрузки данных);
- `city` — город проживания пользователя;
- `tarif` — название тарифного плана.

Пропуски в столбце `churn_date` соответствуют информации по документации, поэтому мы можем их проигнорировать. Хотя в стопцах `reg_date` и `churn_date` сохраняются даты, тип данных у них `object`, в будущем следует это исправить. Типы данных в других столбцах соответствуют своей природе.

In [50]:
calls = pd.read_csv('./datasets/calls.csv')
display(calls.head())
calls.info()

,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
2,1000_2,2018-06-11,2.85,1000
3,1000_3,2018-09-21,13.80,1000
4,1000_4,2018-12-15,5.18,1000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         202607 non-null  object 
 1   call_date  202607 non-null  object 
 2   duration   202607 non-null  float64
 3   user_id    202607 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 6.2+ MB


В таблице `calls` (4 колонки и 202607 строк) сохраняется история звонков для каждого пользователя сети. Столбцы описывают следующие данные:

- `id` — уникальный номер звонка;
- `call_date` — дата звонка;
- `duration` — длительность звонка в минутах;
- `user_id` — идентификатор пользователя, сделавшего звонок.

Пропусков в столбцах нет. Тип данных в столбце `call_date` не отображает действительность, следует его заменить на `datetime64`. Из документации мы знаем, что каждый звонок округляется отдельно: даже если он продлился одну секунду, будет засчитана минута. Поэтому в будущем мы можем привести столбец `duration` к `int64` с округлением вверх.

In [51]:
messages = pd.read_csv('./datasets/messages.csv')
display(messages.head())
messages.info()

,id,message_date,user_id
0,1000_0,2018-06-27,1000
1,1000_1,2018-10-08,1000
2,1000_2,2018-08-04,1000
3,1000_3,2018-06-16,1000
4,1000_4,2018-12-05,1000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123036 entries, 0 to 123035
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            123036 non-null  object
 1   message_date  123036 non-null  object
 2   user_id       123036 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


В таблице `messages` информация о сообщениях пользователей. В общей сложности у нас есть 123036 записей со столбцами:

- `id` — уникальный номер сообщения;
- `message_date` — дата сообщения;
- `user_id` — идентификатор пользователя, отправившего сообщение.

Тип `message_date` следует заменить на `datetime64`.

In [52]:
internet = pd.read_csv('./datasets/internet.csv')
display(internet.head())
internet.info()

,Unnamed: 0,id,mb_used,session_date,user_id
0,0,1000_0,112.95,2018-11-25,1000
1,1,1000_1,1052.81,2018-09-07,1000
2,2,1000_2,1197.26,2018-06-25,1000
3,3,1000_3,550.27,2018-08-22,1000
4,4,1000_4,302.56,2018-09-24,1000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149396 entries, 0 to 149395
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    149396 non-null  int64  
 1   id            149396 non-null  object 
 2   mb_used       149396 non-null  float64
 3   session_date  149396 non-null  object 
 4   user_id       149396 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 5.7+ MB


В таблице `internet` информация об интернет-сессиях. Согласно документации:

- `id` — уникальный номер сессии;
- `mb_used` — объём потраченного за сессию интернет-трафика (в мегабайтах);
- `session_date` — дата интернет-сессии;
- `user_id` — идентификатор пользователя.

Также можно увидеть еще одну колонку `Unnamed: 0`, о которой нет упоминания в документации. Скорее всего, это колонка с индексами, причиной создания которой стало неправильное сохранение `csv` файла. Мы можем удалить эту колонку.
Тип данных в столбце `session_date` не соответствует действительности.

In [53]:
tariffs = pd.read_csv('./datasets/tariffs.csv')
display(tariffs)
tariffs.info()

,messages_included,mb_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,tariff_name
0,50,15360,500,550,200,3,3,smart
1,1000,30720,3000,1950,150,1,1,ultra


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   messages_included      2 non-null      int64 
 1   mb_per_month_included  2 non-null      int64 
 2   minutes_included       2 non-null      int64 
 3   rub_monthly_fee        2 non-null      int64 
 4   rub_per_gb             2 non-null      int64 
 5   rub_per_message        2 non-null      int64 
 6   rub_per_minute         2 non-null      int64 
 7   tariff_name            2 non-null      object
dtypes: int64(7), object(1)
memory usage: 256.0+ bytes


В таблице `tariffs` только две записи, соответствующие описанию двух тарифов. Столбцов в таблице 8:

- `tariff_name` — название тарифа
- `rub_monthly_fee` — ежемесячная абонентская плата в рублях
- `minutes_included` — количество минут разговора в месяц, включённых в абонентскую плату
- `messages_included` — количество сообщений в месяц, включённых в абонентскую плату
- `mb_per_month_included` — объём интернет-трафика, включённого в абонентскую плату (в мегабайтах)
- `rub_per_minute` — стоимость минуты разговора сверх тарифного пакета (например, если в тарифе 100 минут разговора в месяц, то со 101 минуты будет взиматься плата)
- `rub_per_message` — стоимость отправки сообщения сверх тарифного пакета
- `rub_per_gb` — стоимость дополнительного гигабайта интернет-трафика сверх тарифного пакета (1 гигабайт = 1024 мегабайта)

Тип всех колонок соответствует действительности. А данные в столбцах соответствуют описанию тарифов:

**Тариф «Смарт»**
1. Ежемесячная плата: 550 рублей
2. Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика
3. Стоимость услуг сверх тарифного пакета: 1. минута разговора: 3 рубля («Мегалайн» всегда округляет вверх значения минут и мегабайтов. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута); 2. сообщение: 3 рубля; 3. 1 Гб интернет-трафика: 200 рублей.

**Тариф «Ультра»**
1. Ежемесячная плата: 1950 рублей
2. Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика
3. Стоимость услуг сверх тарифного пакета: 1. минута разговора: 1 рубль; 2. сообщение: 1 рубль; 3. 1 Гб интернет-трафика: 150 рублей.

#### Вывод
У нас есть 5 датафреймов с данными из 5 разных `csv-файлов`. У данных почти нет ошибок и несоответствий, но чтобы двигаться дальше, нам нужно их устранить.

**Датафрейм `users`:**
- изменить тип данных в колонке `reg_date`;
- изменить тип данных в колонке `churn_date`.

**Датафрейм `calls`:**
- изменить тип данных в колонке `call_date`;
- изменить тип данных в колонке `duration` на `int64` с округлением вверх.

**Датафрейм `messages`:**
- изменить тип данных в колонке `message_date`.

**Датафрейм `internet`:**
- изменить тип данных в колонке `session_date`;
- удалить столбец `Unnamed`.

**Датафрейм `tariffs`** останется без изменений.

## Предобработка данных


### Замена типа данных
Заменим тип данных для столбцов `reg_date`, `churn_date` в датафрейме `users`:

In [54]:
# так как формат записи даты во всех датафреймах одинаков, создадим отдельную переменную для него

date_format = '%Y-%m-%d'

for col in ['reg_date', 'churn_date']:
    users[col] = pd.to_datetime(
        users[col], format=date_format
    )

Заменим тип данных для столбца `call_date` в датафрейме `calls`. Также округлим данные в столбце `duration` и приведем к `int`:

In [55]:
import numpy as np

calls['call_date'] = pd.to_datetime(
    calls['call_date'], format=date_format
)
calls['duration'] = np.ceil(calls['duration']).astype('int')

Заменим тип данных для столбца `message_date` в датафрейме `messages`:

In [56]:
messages['message_date'] = pd.to_datetime(
    messages['message_date'], format=date_format
)

Заменим тип данных для столбца `session_date` в датафрейме `internet` и удалим лишний столбец.

In [57]:
internet['session_date'] = pd.to_datetime(
    internet['session_date'], format=date_format
)
internet = internet.drop(columns='Unnamed: 0')

Посчитайте для каждого пользователя:
- количество сделанных звонков и израсходованных минут разговора по месяцам;
- количество отправленных сообщений по месяцам;
- объем израсходованного интернет-трафика по месяцам;
- помесячную выручку с каждого пользователя (вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана; прибавьте абонентскую плату, соответствующую тарифному плану).


In [60]:
calls['call_month'] = calls['call_date'].dt.month_name()
calls_per_month = calls.pivot_table(index=['user_id', 'call_month'], values='duration', aggfunc=['sum', 'count'])
calls_per_month.head()

sum    count
                   duration duration
user_id call_month                  
1000    August          408       52
        December        333       46
        July            340       47
        June            172       43
        May             159       22

In [61]:
messages['message_month'] = messages['message_date'].dt.month_name()
messages_per_month = messages.pivot_table(index=['user_id', 'message_month'], values='id', aggfunc='count')
messages_per_month.head()

id
user_id message_month    
1000    August         81
        December       70
        July           75
        June           60
        May            22

In [62]:
internet['session_month'] = internet['session_date'].dt.month_name()
internet_per_month = internet.pivot_table(index=['user_id', 'session_month'], values='mb_used', aggfunc='sum')
internet_per_month.head()

mb_used
user_id session_month          
1000    August         14055.93
        December        9817.61
        July           14003.64
        June           23233.77
        May             2253.49